In [79]:
# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

In [78]:
# Load files and create dataframes in pandas

#2015
happiness_data_2015 = "Resources/2015_Happiness_data.csv"
happiness_df_2015 = pd.read_csv(happiness_data_2015)
# Set Index to Country, rename columns and drop unnecessary
happiness_df_2015 = happiness_df_2015.set_index("Country").rename(columns = {
    "Happiness Rank":"2015_rank", "Happiness Score": "2015_score", "Economy (GDP per Capita)": "2015_GDP", 
    "Family": "2015_family","Health (Life Expectancy)": "2015_life_expect", "Freedom": "2015_freedom", 
    "Generosity": "2015_generosity", "Trust (Government Corruption)": "2015_corruption", 
    "Dystopia Residual": "2015_Dystopia"}).drop(["Region", "Standard Error"], axis = 1)
# happiness_df_2015.to_csv("resources/Happiness_data_clean_2015.csv")

#2016
happiness_data_2016 = "Resources/2016_Happiness_data.csv"
happiness_df_2016 = pd.read_csv(happiness_data_2016)
# Set Index to Country
happiness_df_2016 = happiness_df_2016.set_index("Country")
# Rename columns and drop unnecessary
happiness_df_2016 = happiness_df_2016.rename(columns = {
    "Happiness Rank":"2016_rank", "Happiness Score": "2016_score", "Economy (GDP per Capita)": "2016_GDP", 
    "Family": "2016_family","Health (Life Expectancy)": "2016_life_expect", "Freedom": "2016_freedom", 
    "Generosity": "2016_generosity", "Trust (Government Corruption)": "2016_corruption", 
    "Dystopia Residual": "2016_Dystopia"}).drop(["Region", "Lower Confidence Interval", "Upper Confidence Interval"], axis = 1)
# happiness_df_2016.to_csv("resources/Happiness_data_clean_2016.csv")

#2017
happiness_data_2017 = "Resources/2017_Happiness_data.csv"
happiness_df_2017 = pd.read_csv(happiness_data_2017)
# Set Index to Country, rename columns and drop unnecessary
happiness_df_2017 = happiness_df_2017.set_index("Country").rename(columns = {
    "Happiness.Rank":"2017_rank", "Happiness.Score": "2017_score", "Economy..GDP.per.Capita.": "2017_GDP", 
    "Family": "2017_family","Health..Life.Expectancy.": "2017_life_expect", "Freedom": "2017_freedom", 
    "Generosity": "2017_generosity", "Trust..Government.Corruption.": "2017_corruption", 
    "Dystopia.Residual": "2017_Dystopia"}).drop(["Whisker.high", "Whisker.low"], axis = 1)
# happiness_df_2017.to_csv("resources/Happiness_data_clean_2017.csv")

happiness_df_2017.head()


,2017_rank,2017_score,2017_GDP,2017_family,2017_life_expect,2017_freedom,2017_generosity,2017_corruption,2017_Dystopia
Country,,,,,,,,,
Norway,1,7.537,1.616463,1.533524,0.796667,0.635423,0.362012,0.315964,2.277027
Denmark,2,7.522,1.482383,1.551122,0.792566,0.626007,0.355280,0.400770,2.313707
Iceland,3,7.504,1.480633,1.610574,0.833552,0.627163,0.475540,0.153527,2.322715
Switzerland,4,7.494,1.564980,1.516912,0.858131,0.620071,0.290549,0.367007,2.276716
Finland,5,7.469,1.443572,1.540247,0.809158,0.617951,0.245483,0.382612,2.430182


In [104]:
# Merge Data Frames to give summary data frame across time

merged_1516_df = pd.merge(happiness_df_2015, happiness_df_2016, how="left", on=["Country"])

happiness_summary_df = pd.merge(merged_1516_df, happiness_df_2017, how ="left", on=["Country"])

happiness_summary_df.reset_index(inplace=True)

happiness_summary_df.head()


,Country,2015_rank,2015_score,2015_GDP,2015_family,2015_life_expect,2015_freedom,2015_corruption,2015_generosity,2015_Dystopia,...,2016_Dystopia,2017_rank,2017_score,2017_GDP,2017_family,2017_life_expect,2017_freedom,2017_generosity,2017_corruption,2017_Dystopia
0,Switzerland,1,7.587,1.39651,1.34951,0.94143,0.66557,0.41978,0.29678,2.51738,...,2.69463,4.0,7.494,1.564980,1.516912,0.858131,0.620071,0.290549,0.367007,2.276716
1,Iceland,2,7.561,1.30232,1.40223,0.94784,0.62877,0.14145,0.43630,2.70201,...,2.83137,3.0,7.504,1.480633,1.610574,0.833552,0.627163,0.475540,0.153527,2.322715
2,Denmark,3,7.527,1.32548,1.36058,0.87464,0.64938,0.48357,0.34139,2.49204,...,2.73939,2.0,7.522,1.482383,1.551122,0.792566,0.626007,0.355280,0.400770,2.313707
3,Norway,4,7.522,1.45900,1.33095,0.88521,0.66973,0.36503,0.34699,2.46531,...,2.66465,1.0,7.537,1.616463,1.533524,0.796667,0.635423,0.362012,0.315964,2.277027
4,Canada,5,7.427,1.32629,1.32261,0.90563,0.63297,0.32957,0.45811,2.45176,...,2.70485,7.0,7.316,1.479204,1.481349,0.834558,0.611101,0.435540,0.287372,2.187264


In [121]:
# Create a summary DataFrame 2015-2017
countries = happiness_summary_df["Country"]
summary_df = pd.DataFrame({"Country": countries})
average_rank = []
summary_df["average_score"] = ""
summary_df["average_GDP"]= ""
average_family = ""
average_life_expect = ""
average_freedom = ""
average_corruption = ""
average_generosity = ""
happiness_rank = ""
GDP_bracket = ""

for index,row in countries.iterrows():
    average_rank.append(happiness_summary_df.loc["2015_rank", "2016_rank", "2017_rank"].mean())
    
summary_df = pd.DataFrame({"Country": countries, "average_rank" : average_rank})
summary_df.head()


IndexingError: Too many indexers

In [ ]:
# Target Countries Dataframe (US, UK, Germany, Mexico, Philippines, Singapore)
# 

In [ ]:
# Happiness Data Analysis - Scatter plots
# Function for scatter plot and linear regression
def regression(x_axis, y_axis, title, x_label, y_label):
    slope, y_int, r, pval, stderr = linregress(x_axis, y_axis)
    plt.scatter(x_axis, y_axis)
    plt.title(title)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.grid(b=True, which="major",axis="both", color="lightgrey")
    regression = slope * x_axis + y_int
    plt.plot(x_axis, regression, color = "red")
    plt.show()

# GDP vs. happiness
plt.subplot(2, 2, 1)
regression(summary_df["average_rank"], summary_df["average_GDP"], "Happiness vs GDP 2015-2017", 
           "Happiness Rank", "GDP")

# Freedom vs. happiness

# Generosity vs happiness

# Family vs happiness

# Corruption vs. happiness


In [ ]:
# create bins/Group by for income brackets (4)- add to summary dataframe

In [ ]:
# Box and Whiskers- happiness by income bracket

In [ ]:
# create bins for corruption, create bins (4 each) for happiness add to summary df

In [ ]:
# Heat map- happiness by corruption 